# parsing_tir

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import re
from tqdm.notebook import trange, tqdm
from tqdm.gui import tqdm as tqdm_gui
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display


импорт библиотек

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

reqx = requests.get('https://tiraet.com/catalog/bytovaya/?PAGEN_1=102')

soup = bs(reqx.text, 'html.parser')

element = soup.findAll('div', class_='item_info')

element

title = element[7].find('a', class_='dark_link js-notice-block__title option-font-bold font_sm').text.strip()
title

создаем переменную с наименованием товара

urls = 'https://tiraet.com' + element[7].find('a', class_='dark_link js-notice-block__title option-font-bold font_sm')['href']

counts = element[0].find('div', class_='item-stock js-show-stores js-show-info-block').text.strip()

art = int(element[0].find('div', class_='article_block').text.replace('Арт.: ',''))

price = float(element[7].find('div', class_='price_matrix_wrapper').text.strip().replace(' руб','').replace(',','.').replace(' ',''))

old_price = float(element[7].find('div', class_='price_matrix_wrapper strike_block').text.strip().replace(' руб','').replace(',','.').replace(' ','')) if str(type(element[7].find('div', class_='price_matrix_wrapper strike_block'))) != "<class 'NoneType'>" else 0

datе = pd.to_datetime('today').normalize()

data = []
for i in element:
    data.append({'title' : i.find('a', class_='dark_link js-notice-block__title option-font-bold font_sm').text.strip(),
                 'urls' : 'https://tiraet.com' + i.find('a', class_='dark_link js-notice-block__title option-font-bold font_sm')['href'],  
                 'datе' : pd.to_datetime('today').normalize(),                
                 'counts' : i.find('div', class_='item-stock js-show-stores js-show-info-block').text.strip(),
                'id' : int(i.find('div', class_='article_block').text.replace('Арт.: ','')),
                 'price' : float(i.find('div', class_='price_matrix_wrapper').text.strip().replace(' руб','').replace(',','.').replace(' ','')) if str(type(i.find('div', class_='price_matrix_wrapper'))) != "<class 'NoneType'>" else np.NaN    ,
                 'old_price' : float(i.find('div', class_='price_matrix_wrapper strike_block').text.strip().replace(' руб','').replace(',','.').replace(' ','')) if str(type(i.find('div', class_='price_matrix_wrapper strike_block'))) != "<class 'NoneType'>" else np.NaN                
                })

data = pd.DataFrame(data)
data

In [3]:
def preprocessing(x):
    print(f'таблица имеет следующий вид:')
    display(x.head())
    print('*'*100)
    print(f'таблица имеет {x.shape[0]} сторок и {x.shape[1]} столбцов')
    print('*'*100)
    print(f'в таблице обнаружено дублекатов:{x.duplicated().sum()}')
    print('*'*100)
    print('в таблице обнаружены пропуски в следующих столбцах:')
    display(x.isna().sum())
    print('*'*100)
    print('Эти пропуски составлябт следующее количество в процентах')
    display((x.isna().mean()* 100).round(1))
    print('*'*100)
    print('столбцы имеют следующие типы:')
    display(x.dtypes)
    x.columns = [y.lower().replace(' ', '_') for y in x.columns.values] 
    print('*'*100)
    print('названия столбцов переписаны в нижнем регистре, пробелы заменены на нижнее подчеркивание')
    print('*'*100)
    display(x.info())

создаем функцию предварительного просмотра данных

In [4]:
url = ['https://tiraet.com/catalog/PHoto--i-video/?PAGEN_1=']

In [5]:
url = ['https://tiraet.com/catalog/bytovaya/?PAGEN_1=102'] 

In [6]:
oll_url=['https://tiraet.com/catalog/Komplektujuschie/?PAGEN_1=',
         'https://tiraet.com/catalog/2873/?PAGEN_1=',
         'https://tiraet.com/catalog/Kompjuternaja-mebel/?PAGEN_1=',
         'https://tiraet.com/catalog/bytovaya/?PAGEN_1=',
         'https://tiraet.com/catalog/ohrannye-sistemy/?PAGEN_1=',
         'https://tiraet.com/catalog/svet-i-jelektrika/?PAGEN_1=',
         'https://tiraet.com/catalog/orgtehnika/?PAGEN_1=',
         'https://tiraet.com/catalog/PHoto--i-video/?PAGEN_1=',
         'https://tiraet.com/catalog/setevoe-oborudovanie/?PAGEN_1=',
         'https://tiraet.com/catalog/jelektropitanie/?PAGEN_1=',
         'https://tiraet.com/catalog/uslugi/?PAGEN_1=',
         'https://tiraet.com/catalog/avtojelektronika/?PAGEN_1=',
         'https://tiraet.com/catalog/rashodnye-materialy/?PAGEN_1=',
         
]

In [7]:
j=1
def parsing_2(url):
    data = pd.DataFrame(columns=['title', 'urls', 'datе', 'counts', 'id', 'price', 'old_price'])    
    for u in url:
        reqx = requests.get(u)
        soup = bs(reqx.text, 'html.parser')
        max_page = soup.findAll('div', class_='nums')
        maxx = int(max_page[0].text.strip().split('\n')[-1])
        for j in tqdm(range(1,maxx + 1)):        
            reqx = requests.get(u + str(j))
            soup = bs(reqx.text, 'html.parser')
            element = soup.findAll('div', class_='item_info')
            datax = []
            for i in element:
                datax.append({'title' : i.find('a', class_='dark_link js-notice-block__title option-font-bold font_sm').text.strip(),
                             'urls' : 'https://tiraet.com' + i.find('a', class_='dark_link js-notice-block__title option-font-bold font_sm')['href'],
                             'datе' : pd.to_datetime('today').normalize(),
                             'counts' : i.find('div', class_='item-stock js-show-stores js-show-info-block').text.strip(),
                             'id' : int(i.find('div', class_='article_block').text.replace('Арт.: ','')),
                             'price' : float(i.find('div', class_='price_matrix_wrapper').text.strip().replace(' руб','').replace(',','.').replace(' ','')) if str(type(i.find('div', class_='price_matrix_wrapper'))) != "<class 'NoneType'>" else np.NaN ,
                             'old_price' : float(i.find('div', class_='price_matrix_wrapper strike_block').text.strip().replace(' руб','').replace(',','.').replace(' ','')) if str(type(i.find('div', class_='price_matrix_wrapper strike_block'))) != "<class 'NoneType'>" else np.NaN 
                            })
            datax = pd.DataFrame(datax)
            data = pd.concat([data, datax] , ignore_index=True)
    return data

- 179
- 16
- 23
- 100
- 33
- 60
- 10
- 4
- 31
- 25
- 5
- 5
- 81

In [8]:
df = parsing_2(oll_url)
df

  0%|          | 0/185 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/102 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

...

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

,title,urls,datе,counts,id,price,old_price
0,"HDD 3.5 SAS 3.0, 12Gb/s, 4Tb SEAGATE Exos 7E10 (ST4000NM001B) (7200rpm, 256Mb)",https://tiraet.com/catalog/Komplektujuschie/komplektujuschie/zhestkie-diski/hdd_3_5_sas_3_0_12gb_s_4tb_seagate_exos_7e10_st4000nm001b_7200rpm_256mb/,2024-06-20,Мало,86622,NaN,NaN
1,Переходник для мыши COM в PS/2 (A-235),https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/kabelja-udliniteli-i-perehodniki/Perehodnik-dlja-myshi-COM-v-PS2-A-235/,2024-06-20,Много,284,5.0,NaN
2,"Кабель MINI USB2.0 (A - MINI USB 4PM)1,8м (CC-USB2-AM4P-6)",https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/kabelja-udliniteli-i-perehodniki/kabel-mini-usb20-a---mini-usb-4pm18m-cc-usb2-am4p-6/,2024-06-20,Много,8683,5.0,NaN
3,"BOX для CD/DVD Verbatim Slim, на 1 диск 120мм, PVC (49979) Black 1шт",https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/sumki-dlja-diskov/box_dlya_cd_dvd_verbatim_slim_na_1_disk_120mm_pvc_49979_black_1sht/,2024-06-20,Много,85078,6.6,NaN
4,"Диск CD-R 700MB VERBATIM, 52X (1шт) EXTRA PROTECTION(43411/43351/43437)",https://tiraet.com/catalog/Komplektujuschie/aksessuarycomp/diski/disk_cdr_700mb_verbatim_52x_1sht_extra_protection_43411_43351_43437/,2024-06-20,Много,12819,6.6,NaN
...,...,...,...,...,...,...,...
11581,Головка для плоттеров Canon PF-03 (2251B001),https://tiraet.com/catalog/rashodnye-materialy/zapchasti-dlja-printerov-i-kopirov/zapchasti_print/golovka-dlja-plotterov-canon-pf-03-2251b001/,2024-06-20,Мало,27477,7920.0,NaN
11582,Узел термозакрепления (для сетевых апп)CANON iR2202/2204(FM1-F162-0XX),https://tiraet.com/catalog/rashodnye-materialy/zapchasti-dlja-printerov-i-kopirov/zapchasti_print/uzel_termozakrepleniya_dlya_setevykh_app_canon_ir2202_2204_fm1_f162_0xx/,2024-06-20,Мало,55309,8085.0,NaN
11583,Головка для плоттеров Canon PF-04 (3630B001),https://tiraet.com/catalog/rashodnye-materialy/zapchasti-dlja-printerov-i-kopirov/zapchasti_print/golovka_dlya_plotterov_canon_pf_04_3630b001/,2024-06-20,Мало,56144,8250.0,NaN
11584,Картридж EPSON MX-80/LX-300/400/800/850/880 Patron,https://tiraet.com/catalog/rashodnye-materialy/Rashodnye-k-matrichnym-printeram-kartridzhi-lenty/kartridji_matr/kartridzh-epson-lx-300400800850880mx-80/,2024-06-20,Нет в наличии,249,38.0,NaN


In [9]:
df.to_csv('csv/tiraet_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)  

In [10]:
df = pd.read_csv('csv/tiraet_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv')

In [11]:
tiraet_group = pd.read_csv('csv/atiraet_group.csv', sep = '%')

In [12]:
df = df.merge(tiraet_group, how='left', on='id')

In [13]:
df_new = df[(df['_category'].isna()) | (df['_subcategory'].isna()) | (df['_group'].isna())]

In [14]:
df = df[~((df['_category'].isna()) | (df['_subcategory'].isna()) | (df['_group'].isna()))]

In [15]:
def find_groups(x):
    reqx = requests.get(x)
    soup = bs(reqx.text, 'html.parser')    
    elem_1 = soup.findAll('a', class_='breadcrumbs__link colored_theme_hover_bg-el-svg')
    elem_2 = soup.findAll('div', class_='breadcrumbs__item')
    try:
        if len(elem_1) <=2:
            if len(elem_2) <= 4:
                d_14 = (elem_1[0].text.replace(',','').replace(' ','_').lower())
                d_15 = (elem_1[1].text.replace(',','').replace(' ','_').lower())
                d_16 = (elem_1[1].text.replace(',','').replace(' ','_').lower())
            else:
                d_14 = (elem_1[0].text.replace(',','').replace(' ','_').lower())
                d_15 = (elem_1[1].text.replace(',','').replace(' ','_').lower())
                d_16 = (elem_2[4].text.replace(',','').replace(' ','_').lower())
        else:
            d_14 = (elem_1[0].text.replace(',','').replace(' ','_').lower())
            d_15 = (elem_1[1].text.replace(',','').replace(' ','_').lower())
            d_16 = (elem_1[2].text.replace(',','').replace(' ','_').lower())
    except:
        d_14 = 'err_'
        d_15 = 'err_'
        d_16 = 'err_'
        
    return pd.Series([ (d_14), (d_15), (d_16)])

In [ ]:
tqdm.pandas() 
df_new[['_category','_subcategory','_group']] = df_new['urls'].progress_apply(find_groups)

  0%|          | 0/11586 [00:00<?, ?it/s]

In [ ]:
df_new._category.unique()

In [ ]:
df_new._subcategory.unique()

In [ ]:
df_new._group.unique()

In [ ]:
data = df_new[~(df_new['_category']=='err_')].reset_index(drop=True)
data = data[['id', '_category','_subcategory','_group' ]]
data = data.sort_values(['_category','_subcategory', '_group'])

In [ ]:
tiraet_group = pd.concat([tiraet_group, data], ignore_index=True).sort_values(['_category','_subcategory', '_group']).reset_index(drop=True)
tiraet_group.to_csv('csv/atiraet_group.csv', index=False, sep='%')  

In [ ]:
df = pd.concat([df, df_new], ignore_index=True).sort_values(['_category','_subcategory', '_group']).reset_index(drop=True)

In [ ]:
df.to_csv('csv/tiraet_final_'+dt.datetime.now().strftime('%Y_%m_%d')+'.csv', index=False)  

# Тесты

In [ ]:
reqx = requests.get('https://tiraet.com/catalog/rashodnye-materialy/')

In [ ]:
soup = bs(reqx.text, 'html.parser')

In [ ]:
element2 = soup.findAll('div', class_='inner_wrap TYPE_3')

In [ ]:
element2 = soup.find('span', class_="section-gallery-wrapper flexbox")

In [ ]:
element2[3].find('span', class_="section-gallery-wrapper flexbox").text

In [ ]:
for i in element2:
    print(element2[i].text)


In [ ]:
type(element2)